In [0]:
dbutils.widgets.text('containerName', '', 'cn Name')
dbutils.widgets.text('storageAccountName', '', 'sa')

In [0]:
containerName = dbutils.widgets.get('containerName')
storageAccountName = dbutils.widgets.get('storageAccountName')

In [0]:
configName = f"fs.azure.account.key.{storageAccountName}.dfs.core.windows.net"
bronzeLayerLocation = f"abfss://{containerName}@{storageAccountName}.dfs.core.windows.net/Bronze"
countryLocation = f"abfss://{containerName}@{storageAccountName}.dfs.core.windows.net/Europe/EuropeCountry.csv"
silverLayerLocation = f"abfss://{containerName}@{storageAccountName}.dfs.core.windows.net/Silver"

In [0]:
outputMessage = {"success": 1, "errorMessage": []}

In [0]:
# dbutils.secrets.listScopes()

In [0]:
try:
    accesskey = dbutils.secrets.get('DatabricksKeyVault', 'accesskey')
    spark.conf.set(configName, accesskey)
except Exception as e:
    outputMessage['success'] = 0
    outputMessage['errorMessage'].append(e.desc)

In [0]:
outputMessage

Out[50]: {'success': 0,
 'errorMessage': ['Secret does not exist with scope: DatabricksKeyVault and key: accesske']}

In [0]:
%sql 
drop table if exists InputData

In [0]:
spark.read.parquet(bronzeLayerLocation).createOrReplaceTempView("BronzeLayer")

In [0]:
%sql
--creating managed Input table to do futher transformation.
create table IF NOT EXISTS InputData as
select * from BronzeLayer

num_affected_rows,num_inserted_rows


In [0]:
EuropeCountrydf = spark.read.csv(countryLocation, header=True)
EuropeCountrydf.createOrReplaceTempView('Countries')
EuropeCountrydf.show(2)

+--------------------+-----------+
|         CountryName|CountryCode|
+--------------------+-----------+
|United Kingdom of...|        GBR|
|   Kingdom of Norway|        NOR|
+--------------------+-----------+
only showing top 2 rows



In [0]:
%sql 

drop table if exists SilverLayer;
create table SilverLayer 
using Delta 
as 
  with RefineData as (
  select current_date() as File_date
        , record_id
        , customer_type as customer_type
        , to_date(transaction_date, "dd/MM/yyyy") as transaction_date 
        , to_date(posting_date, "dd/MM/yyyy") as posting_date
        , transaction_amount
        , regexp_replace(trim(trim(trailing country from txn_description)), "(#+)$", '') as Description_text
        , txn_description as original_description_text
        , country as country_code 
        , 1 as EU_flag
        , trim(age_band) as age_band
        , case gender when 'Male' then 'M' when 'Female' then 'F' else 'X' end as gender_code
        , transaction_id
        , customer_key
        , customer_postcode as post_code
        , regexp_replace(Notes, '\n', '') as note_text        
  from InputData
  ) 

  select File_date
         , record_id
         , case when age_band = '>=70' or gender_code = 'X' then "Unspecified" else customer_type end customer_type
         , transaction_date
         , case when datediff(posting_date, transaction_date) >= 1 then posting_date else transaction_date end as posting_date
         , transaction_amount
         , Description_text
         , original_description_text
         , country_code
         , case when c.CountryCode is null then 0
                when c.CountryCode <> 'GBR' then 1 
                when c.CountryCode = 'GBR' and rd.transaction_date <= '2020-01-31' then 1 
                else 0 end  EU_flag
         , age_band
         , gender_code
         , transaction_id
         , customer_key
         , case 
           when country_code = "GBR"  or (country_code is null and regexp_like(post_code, '[A-Za-z]{1,2}[0-9Rr][0-9A-Za-z]? [0-9][ABD-HJLNP-UW-Zabd-hjlnp-uw-z]{2}'))
                 then left(replace(post_code,' ',''), len(post_code)-2) || '**'
           else post_code  end post_code
         , note_text     
  from refineData rd
    left outer join countries c on c.CountryCode = rd.country_code

num_affected_rows,num_inserted_rows


In [0]:
spark.read.table('SilverLayer').write.mode("overwrite").option('overwriteSchema', 'true').format("delta").save(silverLayerLocation)

In [0]:
dbutils.notebook.exit(outputMessage)

Out[54]: <bound method DBUtils.NotebookHandler.exit of Package 'dbutils.notebook'. For more information, type 'dbutils.notebook.help()' in a cell.>